In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from sparkmeasure import StageMetrics
spark=SparkSession \
        .builder \
        .master("spark://ubuntu:7077")\
        .appName("Top 10 users with most ratings every year")\
        .getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [2]:
stagemetrics = StageMetrics(spark)

In [3]:
stagemetrics.begin()
Ratings_Dataframe=(spark.read.format("csv")
            .option("header","True")
            .option("inferSchema","True")
            .load("/home/meli/Documents/Kaggle/rating.csv")
      )
Ratings_Dataframe=Ratings_Dataframe.withColumn("timestamp",date_format("timestamp","yyy"))

df1=(Ratings_Dataframe.groupBy("userId","timestamp").count()
     .withColumnRenamed('count', 'NumberOfRatings')
    )

windowDept = Window.partitionBy("timestamp").orderBy(col("NumberOfRatings").desc())

FullQuery=(df1.withColumn("row",row_number().over(windowDept))
           .filter(col("row") <= 10)
           .drop("row")
           .sort(["timestamp","NumberOfRatings"],ascending=[0,0])
          )

FullQuery.where("timestamp==1995").show()
stagemetrics.end()

+------+---------+---------------+
|userId|timestamp|NumberOfRatings|
+------+---------+---------------+
|131160|     1995|              3|
| 28507|     1995|              1|
+------+---------+---------------+



In [4]:
stagemetrics.print_report()


Scheduling mode = FIFO
Spark Context default degree of parallelism = 4
Aggregated Spark stage metrics:
numStages => 5
numTasks => 413
elapsedTime => 67428 (1.1 min)
stageDuration => 65427 (1.1 min)
executorRunTime => 195210 (3.3 min)
executorCpuTime => 82397 (1.4 min)
executorDeserializeTime => 14700 (15 s)
executorDeserializeCpuTime => 3676 (4 s)
resultSerializationTime => 256 (0.3 s)
jvmGCTime => 2775 (3 s)
shuffleFetchWaitTime => 0 (0 ms)
shuffleWriteTime => 258 (0.3 s)
resultSize => 1437492 (1403.0 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 69206016
recordsRead => 40000528
bytesRead => 1381427650 (1317.0 MB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 4
shuffleTotalBlocksFetched => 4
shuffleLocalBlocksFetched => 4
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 312 (312 Bytes)
shuffleLocalBytesRead => 312 (312 Bytes)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadToDisk => 0 (0 Bytes)